<a href="https://colab.research.google.com/github/filchyboy/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/LS_DS11_213_Ridge_Regression_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [X] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [X] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import category_encoders as ce
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from IPython.display import display, HTML
from sklearn.linear_model import RidgeCV
import matplotlib.pyplot as plt

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

##Narrowed Data Set by Type of Dwelling & Value of Property

In [0]:
df = df[df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS']
df = df[df['SALE_PRICE'] > 100000]
df = df[df['SALE_PRICE'] < 2000000]

In [7]:
df.shape

(3151, 21)

##Clean Data

###Checking Nulls

In [8]:
df.isnull().sum()

BOROUGH                              0
NEIGHBORHOOD                         0
BUILDING_CLASS_CATEGORY              0
TAX_CLASS_AT_PRESENT                 0
BLOCK                                0
LOT                                  0
EASE-MENT                         3151
BUILDING_CLASS_AT_PRESENT            0
ADDRESS                              0
APARTMENT_NUMBER                  3150
ZIP_CODE                             0
RESIDENTIAL_UNITS                    0
COMMERCIAL_UNITS                     0
TOTAL_UNITS                          0
LAND_SQUARE_FEET                     0
GROSS_SQUARE_FEET                    0
YEAR_BUILT                           0
TAX_CLASS_AT_TIME_OF_SALE            0
BUILDING_CLASS_AT_TIME_OF_SALE       0
SALE_PRICE                           0
SALE_DATE                            0
dtype: int64

In [0]:
df = df.drop(['EASE-MENT','APARTMENT_NUMBER','TAX_CLASS_AT_TIME_OF_SALE'], axis=1)

###Datatypes & Mismatches

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3151 entries, 44 to 23035
Data columns (total 18 columns):
BOROUGH                           3151 non-null object
NEIGHBORHOOD                      3151 non-null object
BUILDING_CLASS_CATEGORY           3151 non-null object
TAX_CLASS_AT_PRESENT              3151 non-null object
BLOCK                             3151 non-null int64
LOT                               3151 non-null int64
BUILDING_CLASS_AT_PRESENT         3151 non-null object
ADDRESS                           3151 non-null object
ZIP_CODE                          3151 non-null float64
RESIDENTIAL_UNITS                 3151 non-null float64
COMMERCIAL_UNITS                  3151 non-null float64
TOTAL_UNITS                       3151 non-null float64
LAND_SQUARE_FEET                  3151 non-null object
GROSS_SQUARE_FEET                 3151 non-null float64
YEAR_BUILT                        3151 non-null float64
BUILDING_CLASS_AT_TIME_OF_SALE    3151 non-null object
SALE_PR

####'LAND_SQUARE_FEET' is an object when it should be an integer.

In [11]:
df['LAND_SQUARE_FEET'].head()

44     6,800
61     4,000
78     3,500
108    4,000
111    1,710
Name: LAND_SQUARE_FEET, dtype: object

In [0]:
df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].str.replace(r',', '')
df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].astype(int)

##Set Datetime Format & Split Train/Test by Same


In [13]:
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)
df['SALE_DATE'].describe()

count                    3151
unique                     91
top       2019-01-31 00:00:00
freq                       78
first     2019-01-01 00:00:00
last      2019-04-30 00:00:00
Name: SALE_DATE, dtype: object

In [14]:
df['SALE_DATE'].dt.month.value_counts()


1    947
3    799
2    761
4    644
Name: SALE_DATE, dtype: int64

In [15]:
df = df.set_index('SALE_DATE')
train = df['2019-01':'2019-03'].reset_index()
test  = df['2019-04':].reset_index()
df = df.reset_index()
train.shape, test.shape

((2507, 18), (644, 18))

##Encode

###Drop 'BUILDING_CLASS_CATEGORY'
All data is now aligned on this constant so it can be removed.


In [0]:
train = train.drop(['BUILDING_CLASS_CATEGORY'], axis=1)
test = test.drop(['BUILDING_CLASS_CATEGORY'], axis=1)

In [17]:
train.select_dtypes(include='number').describe().T

,count,mean,std,min,25%,50%,75%,max
BLOCK,2507.0,6758.303949,3975.909029,21.0,3837.5,6022.0,9888.5,16350.0
LOT,2507.0,75.778221,157.531138,1.0,21.0,42.0,70.0,2720.0
ZIP_CODE,2507.0,10993.398484,494.291462,10301.0,10314.0,11234.0,11413.0,11697.0
RESIDENTIAL_UNITS,2507.0,0.987635,0.110532,0.0,1.0,1.0,1.0,1.0
COMMERCIAL_UNITS,2507.0,0.016354,0.129966,0.0,0.0,0.0,0.0,2.0
TOTAL_UNITS,2507.0,1.003989,0.171794,0.0,1.0,1.0,1.0,3.0
LAND_SQUARE_FEET,2507.0,3146.051057,1798.714872,0.0,2000.0,2600.0,4000.0,18906.0
GROSS_SQUARE_FEET,2507.0,1473.744715,599.217635,0.0,1144.0,1368.0,1683.0,7875.0
YEAR_BUILT,2507.0,1944.766653,27.059337,1890.0,1925.0,1940.0,1960.0,2018.0
SALE_PRICE,2507.0,621573.742321,291607.225676,104000.0,440500.0,560000.0,750000.0,1955000.0


In [18]:
test.select_dtypes(include='number').describe().T

,count,mean,std,min,25%,50%,75%,max
BLOCK,644.0,7539.559006,3854.438406,107.0,4500.00,7393.0,10857.5,16350.0
LOT,644.0,76.773292,174.379223,1.0,21.00,41.0,68.0,2202.0
ZIP_CODE,644.0,11158.880435,410.161914,10030.0,11208.00,11362.0,11419.0,11697.0
RESIDENTIAL_UNITS,644.0,0.987578,0.124087,0.0,1.00,1.0,1.0,2.0
COMMERCIAL_UNITS,644.0,0.013975,0.117479,0.0,0.00,0.0,0.0,1.0
TOTAL_UNITS,644.0,1.001553,0.171891,0.0,1.00,1.0,1.0,2.0
LAND_SQUARE_FEET,644.0,2980.018634,1751.056229,0.0,2000.00,2500.0,3800.0,17340.0
GROSS_SQUARE_FEET,644.0,1456.923913,533.476881,0.0,1150.75,1344.0,1685.5,4026.0
YEAR_BUILT,644.0,1939.521739,24.712741,1899.0,1925.00,1930.0,1950.0,2018.0
SALE_PRICE,644.0,655757.054348,296979.954119,107500.0,470000.00,599300.0,781625.0,1912500.0


In [19]:
train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq,first,last
TAX_CLASS_AT_PRESENT,2507,2,1,2476,NaT,NaT
BOROUGH,2507,5,4,1204,NaT,NaT
NEIGHBORHOOD,2507,6,OTHER,2382,NaT,NaT
BUILDING_CLASS_AT_TIME_OF_SALE,2507,11,A1,919,NaT,NaT
BUILDING_CLASS_AT_PRESENT,2507,13,A1,919,NaT,NaT
SALE_DATE,2507,68,2019-01-31 00:00:00,78,2019-01-01,2019-03-30
ADDRESS,2507,2497,216-29 114TH ROAD,2,NaT,NaT


In [20]:
test.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq,first,last
TAX_CLASS_AT_PRESENT,644,2,1,635,NaT,NaT
BOROUGH,644,5,4,376,NaT,NaT
NEIGHBORHOOD,644,6,OTHER,608,NaT,NaT
BUILDING_CLASS_AT_TIME_OF_SALE,644,10,A1,267,NaT,NaT
BUILDING_CLASS_AT_PRESENT,644,11,A1,266,NaT,NaT
SALE_DATE,644,23,2019-04-15 00:00:00,43,2019-04-01,2019-04-30
ADDRESS,644,643,46-12 30TH ROAD,2,NaT,NaT


In [0]:
categories  = ['NEIGHBORHOOD','BOROUGH','BUILDING_CLASS_AT_TIME_OF_SALE','BUILDING_CLASS_AT_PRESENT','TAX_CLASS_AT_PRESENT']

In [22]:
for each in categories:
  print('Category',each,'\n',train[each].value_counts(),'\n\n',
        train.groupby(each)['SALE_PRICE'].mean(),'\n',
        train.groupby(each)['SALE_PRICE'].describe())

Category NEIGHBORHOOD 
 OTHER                 2382
FLUSHING-NORTH          77
FOREST HILLS            17
BOROUGH PARK            12
ASTORIA                 11
BEDFORD STUYVESANT       8
Name: NEIGHBORHOOD, dtype: int64 

 NEIGHBORHOOD
ASTORIA               1.001955e+06
BEDFORD STUYVESANT    6.215972e+05
BOROUGH PARK          1.008917e+06
FLUSHING-NORTH        8.689417e+05
FOREST HILLS          1.210753e+06
OTHER                 6.056645e+05
Name: SALE_PRICE, dtype: float64 
                      count          mean  ...        75%        max
NEIGHBORHOOD                              ...                      
ASTORIA               11.0  1.001955e+06  ...  1180000.0  1600000.0
BEDFORD STUYVESANT     8.0  6.215972e+05  ...   860000.0  1128508.0
BOROUGH PARK          12.0  1.008917e+06  ...   988500.0  1850000.0
FLUSHING-NORTH        77.0  8.689417e+05  ...   930000.0  1695000.0
FOREST HILLS          17.0  1.210753e+06  ...  1490000.0  1876000.0
OTHER               2382.0  6.056645e+05  ..

In [0]:
target = 'SALE_PRICE'
features = ['NEIGHBORHOOD','BOROUGH','BUILDING_CLASS_AT_TIME_OF_SALE',
            'BUILDING_CLASS_AT_PRESENT','TAX_CLASS_AT_PRESENT','BLOCK','LOT',
            'ZIP_CODE','RESIDENTIAL_UNITS','COMMERCIAL_UNITS','TOTAL_UNITS',
            'LAND_SQUARE_FEET','GROSS_SQUARE_FEET','YEAR_BUILT']
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [24]:
X_train.head()

,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A3,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_S0,BUILDING_CLASS_AT_PRESENT_A9,BUILDING_CLASS_AT_PRESENT_A1,BUILDING_CLASS_AT_PRESENT_A5,BUILDING_CLASS_AT_PRESENT_A0,BUILDING_CLASS_AT_PRESENT_A2,BUILDING_CLASS_AT_PRESENT_A3,BUILDING_CLASS_AT_PRESENT_S1,BUILDING_CLASS_AT_PRESENT_A4,BUILDING_CLASS_AT_PRESENT_A6,BUILDING_CLASS_AT_PRESENT_A8,BUILDING_CLASS_AT_PRESENT_B2,BUILDING_CLASS_AT_PRESENT_S0,BUILDING_CLASS_AT_PRESENT_B3,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT
0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,5495,801,11230.0,1.0,0.0,1.0,6800,1325.0,1930.0
1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,7918,72,11427.0,1.0,0.0,1.0,4000,2001.0,1940.0
2,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,4210,19,10461.0,1.0,0.0,1.0,3500,2043.0,1925.0
3,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,5212,69,11226.0,1.0,0.0,1.0,4000,2680.0,1899.0
4,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,7930,121,11203.0,1.0,0.0,1.0,1710,1872.0,1940.0


In [25]:
X_test.head()

,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A3,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_S0,BUILDING_CLASS_AT_PRESENT_A9,BUILDING_CLASS_AT_PRESENT_A1,BUILDING_CLASS_AT_PRESENT_A5,BUILDING_CLASS_AT_PRESENT_A0,BUILDING_CLASS_AT_PRESENT_A2,BUILDING_CLASS_AT_PRESENT_A3,BUILDING_CLASS_AT_PRESENT_S1,BUILDING_CLASS_AT_PRESENT_A4,BUILDING_CLASS_AT_PRESENT_A6,BUILDING_CLASS_AT_PRESENT_A8,BUILDING_CLASS_AT_PRESENT_B2,BUILDING_CLASS_AT_PRESENT_S0,BUILDING_CLASS_AT_PRESENT_B3,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT
0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,5913,878,10471.0,1.0,0.0,1.0,5000,2272.0,1930.0
1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,5488,48,10465.0,1.0,0.0,1.0,2500,720.0,1935.0
2,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,5936,31,11209.0,1.0,0.0,1.0,2880,2210.0,1925.0
3,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,7813,24,11210.0,1.0,0.0,1.0,1305,1520.0,1915.0
4,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,8831,160,11229.0,1.0,0.0,1.0,1800,840.0,1925.0


In [26]:
features = X_train.columns
n = len(features)
print(n)

46


### K is 29, Reduced for memory

In [27]:
selector = SelectKBest(score_func=f_regression, k=4)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)
X_train_selected.shape, X_test_selected.shape
selected_mask = selector.get_support()
all_names = X_train.columns
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print('\nFeatures not selected:')
for name in unselected_names:
    print(name)

Features selected:
BUILDING_CLASS_AT_TIME_OF_SALE_A3
BUILDING_CLASS_AT_PRESENT_A3
LAND_SQUARE_FEET
GROSS_SQUARE_FEET

Features not selected:
NEIGHBORHOOD_OTHER
NEIGHBORHOOD_FLUSHING-NORTH
NEIGHBORHOOD_BEDFORD STUYVESANT
NEIGHBORHOOD_FOREST HILLS
NEIGHBORHOOD_BOROUGH PARK
NEIGHBORHOOD_ASTORIA
BOROUGH_3
BOROUGH_4
BOROUGH_2
BOROUGH_5
BOROUGH_1
BUILDING_CLASS_AT_TIME_OF_SALE_A9
BUILDING_CLASS_AT_TIME_OF_SALE_A1
BUILDING_CLASS_AT_TIME_OF_SALE_A5
BUILDING_CLASS_AT_TIME_OF_SALE_A0
BUILDING_CLASS_AT_TIME_OF_SALE_A2
BUILDING_CLASS_AT_TIME_OF_SALE_S1
BUILDING_CLASS_AT_TIME_OF_SALE_A4
BUILDING_CLASS_AT_TIME_OF_SALE_A6
BUILDING_CLASS_AT_TIME_OF_SALE_A8
BUILDING_CLASS_AT_TIME_OF_SALE_S0
BUILDING_CLASS_AT_PRESENT_A9
BUILDING_CLASS_AT_PRESENT_A1
BUILDING_CLASS_AT_PRESENT_A5
BUILDING_CLASS_AT_PRESENT_A0
BUILDING_CLASS_AT_PRESENT_A2
BUILDING_CLASS_AT_PRESENT_S1
BUILDING_CLASS_AT_PRESENT_A4
BUILDING_CLASS_AT_PRESENT_A6
BUILDING_CLASS_AT_PRESENT_A8
BUILDING_CLASS_AT_PRESENT_B2
BUILDING_CLASS_AT_PRESENT_S

###29 Features Seem to be the Sweet Spot

In [28]:
for k in range(1, len(X_train.columns)+1):
    print(f'{k} features')

    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test Mean Absolute Error: ${mae:,.0f} \n')

1 features
Test Mean Absolute Error: $183,641 

2 features
Test Mean Absolute Error: $179,555 

3 features
Test Mean Absolute Error: $179,291 

4 features
Test Mean Absolute Error: $179,291 

5 features
Test Mean Absolute Error: $170,483 

6 features
Test Mean Absolute Error: $169,982 

7 features
Test Mean Absolute Error: $168,140 

8 features
Test Mean Absolute Error: $168,245 

9 features
Test Mean Absolute Error: $167,855 

10 features
Test Mean Absolute Error: $164,737 

11 features
Test Mean Absolute Error: $165,346 

12 features
Test Mean Absolute Error: $164,860 

13 features
Test Mean Absolute Error: $155,159 

14 features
Test Mean Absolute Error: $156,541 

15 features
Test Mean Absolute Error: $156,572 

16 features
Test Mean Absolute Error: $156,573 

17 features
Test Mean Absolute Error: $156,394 

18 features
Test Mean Absolute Error: $156,394 

19 features
Test Mean Absolute Error: $156,255 

20 features
Test Mean Absolute Error: $156,255 

21 features
Test Mean Absolut

In [0]:
alphas = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
ridge = RidgeCV(alphas=alphas, normalize=True)

In [33]:
for alpha in [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]:
    feature = 'GROSS_SQUARE_FEET'
    ridge = RidgeCV(alphas=alphas, normalize=True)
    ridge.fit(X_train[[feature]], y_train)
    ridge.alpha_  
    # Fit Ridge Regression model
  
    display(HTML(f'Ridge Regression, with alpha={alpha}'))
    model = RidgeCV(alphas=alphas, normalize=True)
    model.fit(X_train[[feature]], y_train)
    
    # Get Test MAE
    y_pred = model.predict(X_test[[feature]])
    mae = mean_absolute_error(y_test, y_pred)
    display(HTML(f'Test Mean Absolute Error: ${mae:,.0f}'))
    
    train.plot.scatter(feature, target, alpha=0.05)
    plt.plot(X_test[feature], y_pred)
    plt.show()

In [35]:
for alpha in [0.001, 0.01, 0.1, 1.0, 1, 100.0, 1000.0]:
    ridge = RidgeCV(alphas=alphas, normalize=True)
    ridge.fit(X_train[[feature]], y_train)
    ridge.alpha_  
    display(HTML(f'Ridge Regression, with alpha={alpha}'))
    model = RidgeCV(alphas=alphas, normalize=True)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Get Test MAE
    mae = mean_absolute_error(y_test, y_pred)
    display(HTML(f'Test Mean Absolute Error: ${mae:,.0f}'))
    #print(f'Test Mean Absolute Error: ${mae:,.0f}')
    
    # Plot coefficients
    coefficients = pd.Series(model.coef_, X_train.columns)
    plt.figure(figsize=(16,8))
    coefficients.sort_values().plot.barh(color='grey')
    plt.xlim(-10000,10000)
    plt.show()